## Iterative Financial Research Agent with AgentUp

### Overview

This notebook walks you through the process of building a financial research agent with AgentUp, powered by the
`agentup_systools` and `agentup_brave` plugins.

Instead of stopping at a single query, this agent learns to dig deeper—gathering information from multiple sources, refining its
analysis step by step, and storing insights locally for later use.

Along the way, you’ll see how the agent can search markets in real time—whether that’s equities, crypto, or broader financial
trends—then sharpen its conclusions as it processes the results.

Because all findings can be saved as structured files, the workflow balances immediate insights with a persistent knowledge base.
The result is a system that not only reacts but also builds a record of its reasoning.

By working through the notebook, you’ll gain practical experience with:

- The plugin architecture that allows AgentUp to expand its capabilities
- Designing iterative research loops that improve with each cycle
- Setting up the necessary security scopes and API keys to keep things safe and controlled
- Orchestrating multiple plugins so they function as a unified agent

And finally, shaping these techniques into production-ready applications

Think of this as both a hands-on tutorial and a foundation for building more advanced, domain-specific research agents.

### About AgentUp

[AgentUp](https://github.com/RedDotRocket/AgentUp) is the framework every forward-looking developer will love - just as Docker redefined application deployment, AgentUp lets you define AI agents with simple config and run them anywhere, instantly. It’s built by engineers who crafted mission-critical software for Google, GitHub, Nvidia, Shopify and Red Hat, so it scales, secures, and simplifies agent development with enterprise-grade features right out of the box, and a plugin ecosystem that carries your security, middleware, and CI/CD pipelines for you—if you’re not already learning it, you’re letting the future of AI agent infrastructure pass you by.


## Prerequisites and Setup

Before running this notebook, ensure you have:
1. **Brave Search API key** (get from https://api.search.brave.com/) - there is a good free tier available, no credit card required.
2. **OpenAI, Anthropic API Keys** or.
3. **Ollama** configured!

### Get Required files (Colab only)

Note: This section only need be run if using Google Colab

In [ ]:
import sys
import os
import subprocess

def detect_environment():
    """Detect the current runtime environment and return appropriate project path."""
    try:
        import google.colab
        return 'colab', '/content/proj-folder'
    except ImportError:
        return 'local', '.'

# Environment setup
ENV_TYPE, PROJ_PATH = detect_environment()
print(f"Detected environment: {ENV_TYPE}")
print(f"Project path: {PROJ_PATH}")

if ENV_TYPE == 'colab':
    print("Setting up Google Colab environment...")

    # Clone the repository if it doesn't exist
    if not os.path.exists(PROJ_PATH):
        print("Cloning agentup-financial-research repository...")
        subprocess.run([
            'git', 'clone',
            'https://github.com/RedDotRocket/agentup-financial-research',
            'proj-folder'
        ], check=True)
        print("Repository cloned to proj-folder")
    else:
        print("Project folder already exists")

    # Add the repo to Python path
    if PROJ_PATH not in sys.path:
        sys.path.insert(0, PROJ_PATH)
        print(f"Added {PROJ_PATH} to Python path")

    # Change working directory
    os.chdir(PROJ_PATH)
    print(f"Changed working directory to: {PROJ_PATH}")

else:
    print("Local environment detected - using current directory")
    print(f"Current working directory: {os.getcwd()}")

print("Environment setup complete!")
print(f"Working directory: {os.getcwd()}")
print(f"Python path includes project: {os.getcwd() in sys.path}")

# Make the function globally available for other cells
globals()['detect_environment'] = detect_environment

### Install Dependencies

Install the required Python packages using pip.

If in Google Colab, we use --quiet mode which prevents the Kernel from being unnecessarily restarted, for a transient dependency 'psutil'.

In [ ]:
# Environment-aware installation with quiet mode for Colab
try:
    ENV_TYPE, PROJ_PATH = detect_environment()
    print(f"Using project path: {PROJ_PATH}")

    # Set pip flags based on environment
    if ENV_TYPE == 'colab':
        pip_flags = "--quiet"
        print("Google Colab detected - using quiet installation mode")
    else:
        pip_flags = ""
        print("Local environment - using standard installation mode")

except NameError:
    print("Warning: detect_environment() not found. Run the Environment Setup cell first.")
    # Fallback detection
    try:
        import google.colab
        PROJ_PATH = "/content/proj-folder"
        pip_flags = "--quiet"
        print("Fallback: Google Colab detected - using quiet installation mode")
    except ImportError:
        PROJ_PATH = "."
        pip_flags = ""
        print("Fallback: Local environment - using standard installation mode")

print(f"Using project path: {PROJ_PATH}")
print("Installing AgentUp and dependencies...")

# Install AgentUp and plugins
%pip install {pip_flags} agentup

# Additional dependencies and AgentUp plugins
%pip install {pip_flags} -r {PROJ_PATH}/requirements.txt
%pip install {pip_flags} -r {PROJ_PATH}/agentup-requirements.txt

print("Installation complete!")

### Check plugin status

Check the plugins are loaded and ready.

In [ ]:
!agentup plugin list -c

### Setup Model Provider and Brave Search API Key

The following keys are required:

- **Brave Search API**: Free tier available at [Brave Search API](https://api.search.brave.com/)
- **AI Provider Options**:
  - **OpenAI API**: Get your key from [OpenAI Platform](https://platform.openai.com/api-keys) - Excellent tool calling
  - **Anthropic API**: Get your key from [Anthropic Console](https://console.anthropic.com/) - Superior reasoning
  - **Ollama**: Local deployment at [Ollama.ai](https://ollama.ai/) - ⚠️ Requires models capable of tool calling

**💡 If running locally**: Set environment variables before starting Jupyter:
```bash
# Required for all configurations
export BRAVE_API_KEY='your-brave-api-key-here'

# Choose your AI provider (pick one):
export OPENAI_API_KEY='your-openai-api-key-here'        # Option 1: OpenAI (default)
export ANTHROPIC_API_KEY='your-anthropic-api-key-here'  # Option 2: Anthropic  
export OLLAMA_BASE_URL='http://localhost:11434'         # Option 3: Ollama

jupyter lab
```

In [ ]:
import os
import getpass

def mask_key(key, show_first=4, show_last=4):
    """Mask an API key for display, showing only first and last characters."""
    if not key or len(key) < 8:
        return '***'
    return f"{key[:show_first]}{'*' * (len(key) - show_first - show_last)}{key[-show_last:]}"

# Enhanced environment setup supporting multiple AI providers
def setup_environment():
    """Set up API keys with multi-provider support and masked input."""

    # Check required keys
    missing_keys = []
    if not os.getenv('BRAVE_API_KEY'):
        missing_keys.append('BRAVE_API_KEY')

    # AI Provider Selection
    ai_provider = None
    ai_key_required = None

    # Check if any AI provider is already configured
    existing_providers = []
    if os.getenv('OPENAI_API_KEY'):
        existing_providers.append('OpenAI')
    if os.getenv('ANTHROPIC_API_KEY'):
        existing_providers.append('Anthropic')
    if os.getenv('OLLAMA_BASE_URL'):
        existing_providers.append('Ollama')

    if missing_keys or not existing_providers:
        print("API Key Configuration Required")
        print("=" * 50)

        if existing_providers:
            print(f"✅ Detected existing providers: {', '.join(existing_providers)}")

        if missing_keys:
            print("Missing environment variables:", ', '.join(missing_keys))

        # AI Provider Selection if none configured
        if not existing_providers:
            print("\nAI Provider Selection:")
            print("We need to choose which AI provider to use for this agent.")

            try:
                # VSCode-friendly input with options in the prompt
                ai_choice = input("\nChoose AI Provider: (1) OpenAI GPT-4o-mini [reliable] (2) Anthropic Claude-3.5-Sonnet [reasoning] (3) Ollama [local/private] (4) Skip - Enter 1-4: ").strip()

                if ai_choice == "1":
                    ai_provider = "openai"
                    ai_key_required = "OPENAI_API_KEY"
                    print("✅ Selected: OpenAI (GPT-4o-mini) - Excellent tool calling capabilities")
                elif ai_choice == "2":
                    ai_provider = "anthropic"
                    ai_key_required = "ANTHROPIC_API_KEY"
                    print("✅ Selected: Anthropic (Claude 3.5 Sonnet) - Superior reasoning")
                elif ai_choice == "3":
                    ai_provider = "ollama"
                    ai_key_required = "OLLAMA_BASE_URL"
                    print("✅ Selected: Ollama (Local deployment)")
                    print("⚠️ WARNING: Use 30B+ models (llama3.1:70b, qwen2.5:32b, mixtral:8x7b)")
                    print("Small models (8b) often fail with tools!")
                elif ai_choice == "4":
                    ai_provider = None
                    print("Skipping AI provider setup")
                else:
                    print("Invalid choice, defaulting to OpenAI")
                    ai_provider = "openai"
                    ai_key_required = "OPENAI_API_KEY"

            except (KeyboardInterrupt, EOFError):
                print("\nDefaulting to OpenAI configuration")
                ai_provider = "openai"
                ai_key_required = "OPENAI_API_KEY"

        # Get required keys with masked input
        try:
            # Brave Search API Key (always required)
            if 'BRAVE_API_KEY' in missing_keys:
                print("\nBRAVE SEARCH API KEY REQUIRED")
                print("   Purpose: Enables internet search for financial data")
                print("   Get free key at: https://api.search.brave.com/")
                print("   Sign up → Create App → Copy API key")

                value = getpass.getpass("\n   Enter your Brave Search API key (input will be hidden): ").strip()

                if value:
                    os.environ['BRAVE_API_KEY'] = value
                    print(f"   ✅ BRAVE_API_KEY configured successfully: {mask_key(value)}")
                else:
                    print("   ❌ BRAVE_API_KEY cannot be empty")
                    return False, None

            # AI Provider Key
            if ai_key_required and not os.getenv(ai_key_required):
                if ai_provider == "openai":
                    print("\nOPENAI API KEY REQUIRED")
                    print("   Purpose: Powers the AI agent (GPT-4o-mini model)")
                    print("   Get key at: https://platform.openai.com/api-keys")
                    print("   Login → Create new secret key → Copy key")
                    value = getpass.getpass("\n   Enter your OpenAI API key (input will be hidden): ").strip()

                elif ai_provider == "anthropic":
                    print("\nANTHROPIC API KEY REQUIRED")
                    print("   Purpose: Powers the AI agent (Claude 3.5 Sonnet)")
                    print("   Get key at: https://console.anthropic.com/")
                    print("   Login → API Keys → Create Key → Copy key")
                    value = getpass.getpass("\n   Enter your Anthropic API key (input will be hidden): ").strip()

                elif ai_provider == "ollama":
                    print("\nOLLAMA CONFIGURATION")
                    print("   Purpose: Local AI model deployment")
                    print("   Default: http://localhost:11434")
                    print("   Make sure Ollama is running with a large model!")
                    value = input("\n   Enter Ollama base URL [http://localhost:11434] or press Enter for default: ").strip()
                    if not value:
                        value = "http://localhost:11434"

                if value:
                    os.environ[ai_key_required] = value
                    if ai_provider == "ollama":
                        print(f"   ✅ {ai_key_required} configured successfully: {value}")
                    else:
                        print(f"   ✅ {ai_key_required} configured successfully: {mask_key(value)}")
                else:
                    print(f"   ❌ {ai_key_required} cannot be empty")
                    return False, None

        except (KeyboardInterrupt, EOFError):
            print("\n❌ Setup cancelled by user")
            return False, None
        except Exception as e:
            print(f"\n❌ Error during setup: {e}")
            return False, None

    # Verify configuration with masked display
    brave_key = os.getenv('BRAVE_API_KEY', '')

    # Check which AI provider is configured
    openai_key = os.getenv('OPENAI_API_KEY', '')
    anthropic_key = os.getenv('ANTHROPIC_API_KEY', '')
    ollama_url = os.getenv('OLLAMA_BASE_URL', '')

    configured_provider = None
    if openai_key:
        configured_provider = "openai"
    elif anthropic_key:
        configured_provider = "anthropic"
    elif ollama_url:
        configured_provider = "ollama"

    if brave_key and (configured_provider or ai_provider is None):
        print("\n✅ Environment configured successfully!")
        print("API Keys Status:")
        print(f"   BRAVE_API_KEY: {mask_key(brave_key)}")

        if configured_provider == "openai":
            print(f"   OPENAI_API_KEY: {mask_key(openai_key)}")
            print("   AI Provider: OpenAI (GPT-4o-mini)")
        elif configured_provider == "anthropic":
            print(f"   ANTHROPIC_API_KEY: {mask_key(anthropic_key)}")
            print("   AI Provider: Anthropic (Claude 3.5 Sonnet)")
        elif configured_provider == "ollama":
            print(f"   OLLAMA_BASE_URL: {ollama_url}")
            print("   AI Provider: Ollama (Local deployment)")
            print("   ⚠️ Ensure your model supports tool calling (30B+ recommended)")

        return True, configured_provider or ai_provider
    else:
        print("\n❌ Setup incomplete - missing required keys")
        return False, None

print("=" * 50)

success, provider = setup_environment()
if success:
    print(f"\nReady for AgentUp tutorial with {provider or 'configured'} provider!")
    print("You can now proceed to the next cell.")
else:
    print("\n⚠️ Environment setup needed")


## Agent Configuration

We'll configure our financial research agent with the necessary plugins and security scopes. The configuration follows AgentUp's declarative approach where plugins declare their requirements.

### Understanding Plugin Architecture

- **agentup_systools**: Provides file operations, directory management, and command execution
- **agentup_brave**: Enables web search capabilities through Brave Search API
- **Security Scopes**: Fine-grained permissions (`files:read`, `files:write`, `api:read`, `system:read`)

In [ ]:
# Interactive AgentUp Configuration (Real AgentUp Format)
from jinja2 import Template
from pathlib import Path
from agent_configs import AGENT_CONFIGS

def show_agent_options():
    """Display available agent configurations"""
    print("🤖 Available AgentUp Configurations")
    print("=" * 50)
    for key, config in AGENT_CONFIGS.items():
        print(f"\n📋 {key.upper()}")
        print(f"   Name: {config['name']}")
        print(f"   Description: {config['description']}")
        print(f"   System Prompt: {len(config['system_prompt'])} characters")
    print()

def get_user_choice():
    """Get user's agent choice"""
    print("Which agent would you like to configure?")
    print("1. Financial Research Agent (recommended)")
    print("2. Weather Analysis Agent")
    print("3. Technical Support Agent")
    print("4. Keep existing configuration")

    try:
        choice = input("\nEnter choice (1-4): ").strip()
        choice_map = {"1": "financial", "2": "weather", "3": "technical", "4": "existing"}
        return choice_map.get(choice, "financial")
    except (KeyboardInterrupt, EOFError):
        print("\nUsing default financial configuration")
        return "financial"

def create_agentup_yaml_from_template():
    """Create AgentUp YAML using real AgentUp format"""

    # Show options and get choice
    show_agent_options()
    choice = get_user_choice()

    if choice == "existing":
        print("\n✅ Keeping existing agentup.yml configuration")
        return "existing"

    selected_config = AGENT_CONFIGS[choice]
    print(f"\n✅ Selected: {selected_config['name']}")

    # Load the Jinja2 template
    template_path = Path("./agentup.yml.j2")
    if not template_path.exists():
        print(f"❌ Template file not found: {template_path}")
        return "error"

    with open(template_path, 'r') as f:
        template_content = f.read()

    # Create Jinja2 template
    template = Template(template_content)

    # Render the template with selected configuration
    rendered_yaml = template.render(
        name=selected_config['name'],
        description=selected_config['description'],
        system_prompt=selected_config['system_prompt']
    )

    # Write the rendered YAML
    config_path = Path("./agentup.yml")
    with open(config_path, 'w') as f:
        f.write(rendered_yaml)

    print("✅ AgentUp configuration generated from template")
    print(f"📁 Template: {template_path}")
    print(f"📁 Output: {config_path}")
    print("🧩 System prompt: {len(selected_config['system_prompt'])} characters")
    print("📐 Format: Real AgentUp YAML structure")
    print("🎯 Features: agent_type=iterative, memory_config, iterative_config")

    return config_path

# Install Jinja2 if not available
try:
    from jinja2 import Template
    print("✅ Jinja2 available")
except ImportError:
    print("📦 Installing Jinja2...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "jinja2"])
    print("✅ Jinja2 installed")
    from jinja2 import Template

# Run the configuration setup
print("🚀 AgentUp Configuration Setup (Real AgentUp Format)")
print("=" * 65)

config_result = create_agentup_yaml_from_template()

if config_result != "existing" and config_result != "error":
    print("\n Ready to use AgentUp!")
    print("📁 Configuration file: {config_result}")
elif config_result == "existing":
    print("\n Using existing agentup.yml configuration")
else:
    print("\n❌ Configuration setup failed")

### Setup Research Directories and Conversation States

We need to ensure that the necessary directories for research data and conversation states are set up properly. This will help in organizing our files and maintaining a clean project structure.


In [ ]:
# Use environment-aware paths for directory creation
from pathlib import Path
try:
    ENV_TYPE, PROJ_PATH = detect_environment()
    base_path = Path(PROJ_PATH)
    print(f"Using base path: {base_path}")
except NameError:
    print("Warning: detect_environment() not found. Using current directory.")
    base_path = Path(".")

# Create research data directory for our agent
research_dir = base_path / "research_data"
research_dir.mkdir(exist_ok=True)
print(f"✓ Research directory created: {research_dir}")

# Create conversation states directory for AgentUp state management
states_dir = base_path / "conversation_states"
states_dir.mkdir(exist_ok=True)
print(f"✓ Conversation states directory created: {states_dir}")

# Verify AgentUp configuration exists
config_path = base_path / "agentup.yml"
if config_path.exists():
    print(f"✓ AgentUp configuration ready: {config_path}")
    print(f"   Size: {config_path.stat().st_size} bytes")
else:
    print(f"❌ AgentUp configuration not found: {config_path}")
    print("   Please run the previous cell to generate the configuration")

print(f"Working directory: {Path.cwd()}")
print(f"Research data will be saved to: {research_dir.absolute()}")

## Start the AgentUp Server

Now that our plugins and configuration file is created, we can start the AgentUp server to begin using our financial research agent. Run the following command in your terminal:

This will start the server with the specified configuration, allowing you to interact with the agent and perform research tasks.


In [ ]:
import threading
import subprocess
import time
import sys

# Google Colab port forwarding (graceful fallback)
try:
    import google.colab
    from google.colab import output
    output.serve_kernel_port_as_window(8000)
    print("Google Colab detected - port 8000 forwarding enabled")
except ImportError:
    print("Standard Jupyter environment - port forwarding not needed")

def run_server():
    """Run the AgentUp server in a background thread."""
    try:
        # Use the config file path
        config_path = "./agentup.yml"

        # Start the server without capturing output (let it run freely)
        subprocess.Popen([
            "agentup", "run",
            "--config", config_path,
            "--no-reload"
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

        print("Server process started")

    except Exception as e:
        print(f"❌ Failed to start server: {e}")

# Start server in background thread
print("Starting AgentUp Server in Background Thread...")
print("=" * 50)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Give the server time to start
print("Waiting for server to initialize...")
for i in range(10):
    time.sleep(1)
    print(f"   Starting... {i+1}/10", end="\r")

print("\nServer should be running in background!")
print("Server URL: http://localhost:8000")
print("API Key: research-agent-key-123")
print("\n⚠️  Note: If the server fails to start, check the AgentUp configuration")
print("   and ensure all required API keys are set in environment variables.")
print("\nYou can now proceed to test the agent connectivity in the next cell!")

## Agent-to-Agent (A2A) Communication

AgentUp uses the A2A protocol for communication. We'll create helper functions to interact with our agent using JSON-RPC format.

### Understanding A2A Protocol

The A2A protocol enables standardized communication between agents with:
- **JSON-RPC 2.0** format for requests
- **Message-based** communication with structured parts
- **Security-first** design with authentication headers

In [ ]:
import aiohttp
import asyncio
import json
import time
from typing import Any, Optional, Callable

class FinancialResearchAgent:
    """Client for interacting with the Financial Research Agent via A2A streaming protocol."""

    def __init__(self, base_url: str = "http://localhost:8000", api_key: str = "research-agent-key-123"):
        self.base_url = base_url
        self.api_key = api_key
        self.headers = {
            "Content-Type": "application/json",
            "X-API-Key": api_key
        }
        self.message_counter = 0

    def _create_stream_message(self, text: str, message_id: Optional[str] = None) -> dict[str, Any]:
        """Create a properly formatted A2A message for streaming."""
        if message_id is None:
            message_id = f"msg-{int(time.time())}-{self.message_counter}"
            self.message_counter += 1

        return {
            "jsonrpc": "2.0",
            "method": "message/stream",
            "params": {
                "message": {
                    "role": "user",
                    "parts": [
                        {
                            "kind": "text",
                            "text": text
                        }
                    ],
                    "message_id": message_id,
                    "kind": "message"
                }
            },
            "id": f"req-{message_id}"
        }

    def _parse_streaming_response(self, line: str) -> tuple[Optional[str], bool]:
        """Parse a streaming response line and extract readable content."""
        try:
            # Remove 'data: ' prefix if present
            if line.startswith('data: '):
                line = line[6:]

            chunk_data = json.loads(line)

            if 'result' in chunk_data:
                result = chunk_data['result']

                # Handle different message types
                if result.get('kind') == 'status-update':
                    status = result.get('status', {})
                    message = status.get('message', {})
                    parts = message.get('parts', [])

                    if parts:
                        text = parts[0].get('text', '')
                        state = status.get('state', '')

                        # Format status updates nicely
                        if state == 'working' and text:
                            return f"🔄 {text}\n", False
                        elif text:
                            return f"📝 {text}\n", False

                elif result.get('kind') == 'message':
                    # Final message content
                    parts = result.get('parts', [])
                    if parts:
                        content = parts[0].get('text', '')
                        return content, True

                elif 'content' in result:
                    # Direct content
                    return result['content'], result.get('final', False)

            elif 'error' in chunk_data:
                return f"❌ Error: {chunk_data['error']}\n", True

        except json.JSONDecodeError:
            # Non-JSON line, might be plain text
            if line.strip():
                return f"{line}\n", False
        except Exception as e:
            return f"⚠️ Parse error: {e}\n", False

        return None, False

    async def send_message_stream(self, text: str, on_chunk: Optional[Callable] = None) -> dict[str, Any]:
        """Send a message to the agent and return the streaming response."""
        message = self._create_stream_message(text)

        try:
            timeout = aiohttp.ClientTimeout(total=300)  # 5 minutes for long research
            async with aiohttp.ClientSession(timeout=timeout, headers=self.headers) as session:
                async with session.post(self.base_url, json=message) as response:
                    response.raise_for_status()

                    full_content = ""
                    final_result = {}
                    _is_complete = False

                    async for chunk in response.content.iter_chunked(1024):
                        if chunk:
                            try:
                                chunk_text = chunk.decode('utf-8')

                                # Process each line separately
                                for line in chunk_text.strip().split('\n'):
                                    if not line.strip():
                                        continue

                                    content, is_final = self._parse_streaming_response(line)

                                    if content:
                                        full_content += content

                                        # Call callback for real-time display
                                        if on_chunk:
                                            await on_chunk(content)

                                        # Check if this is the final response
                                        if is_final:
                                            final_result = {
                                                "success": True,
                                                "result": {
                                                    "content": content,
                                                    "role": "assistant"
                                                }
                                            }

                            except UnicodeDecodeError:
                                continue

                    # Return final result
                    if final_result:
                        return final_result
                    else:
                        return {
                            "success": True,
                            "result": {
                                "content": full_content,
                                "role": "assistant"
                            }
                        }

        except Exception as e:
            return {"error": f"Streaming request failed: {str(e)}", "success": False}

    async def send_message(self, text: str) -> dict[str, Any]:
        """Send a message to the agent (backwards compatibility)."""
        return await self.send_message_stream(text)

    async def research_topic_stream(self, topic: str) -> dict[str, Any]:
        """Research a financial topic with iterative analysis and streaming display."""
        prompt = f"""
        Perform comprehensive financial research on: {topic}

        Please follow this iterative process:
        1. Search for recent news and data about {topic}
        2. Analyze the search results for key trends and insights
        3. Create a detailed research report which includes all of your findings.
        4. The report should be at least 3000 words + and explain your research process and how it led to specific findings.
        4. Save the report to a file in the research_data directory unless in google colab, and then save to content/proj-folder/research_data/
        5. Provide a summary of your findings

        Focus on:
        - Recent market movements and trends
        - Key financial metrics and data points
        - Expert opinions and analysis (provide quotes, links)
        - Potential risks and opportunities

        Please use available tools and capabilities to search internet read or write file operations to complete the research.
        """

        print(f"🔍 Starting research on: {topic}")
        print("-" * 60)

        # Create a callback to display clean streaming content
        async def display_chunk(content):
            print(content, end='', flush=True)

        result = await self.send_message_stream(prompt, display_chunk)
        print(f"\n{'-' * 60}")
        print("✅ Research completed!")

        return result

    def research_topic(self, topic: str) -> dict[str, Any]:
        """Synchronous wrapper for research_topic_stream."""
        # Check if we're in a notebook environment
        try:
            from IPython.core.getipython import get_ipython
            if get_ipython() is not None:
                # We're in Jupyter, use the event loop
                loop = asyncio.get_event_loop()
                if loop.is_running():
                    import nest_asyncio
                    nest_asyncio.apply()
                return loop.run_until_complete(self.research_topic_stream(topic))
            else:
                return asyncio.run(self.research_topic_stream(topic))
        except RuntimeError:
            # Handles "There is no current event loop in thread" or similar
            return asyncio.run(self.research_topic_stream(topic))
        except ImportError:
            # Handles missing IPython or nest_asyncio
            return asyncio.run(self.research_topic_stream(topic))

print("✓ Financial Research Agent client created and ready to use!")

### Test Agent Connectivity

Let's test that our agent is running and can respond to basic requests.

In [ ]:
import requests
from typing import Any


def fetch_agent_card(base_url: str = "http://127.0.0.1:8000") -> dict[str, Any]:
    """Fetch the agent card from the well-known endpoint."""
    agent_card_url = f"{base_url}/.well-known/agent-card.json"

    try:
        response = requests.get(agent_card_url, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to fetch agent card: {e}")
    except json.JSONDecodeError as e:
        raise Exception(f"Invalid JSON response: {e}")


def display_agent_info(agent_card: dict[str, Any]) -> None:
    """Display basic agent information."""
    print("✅ Agent Card Retrieved Successfully!")
    print(f"Agent Name: {agent_card.get('name', 'Unknown')}")
    print(f"Description: {agent_card.get('description', 'No description')}")
    print(f"URL: {agent_card.get('url', 'No URL')}")
    print(f"Protocol Version: {agent_card.get('protocolVersion', 'Unknown')}")
    print(f"AgentUp Version: {agent_card.get('version', 'Unknown')}")


def display_capabilities(capabilities: dict[str, Any]) -> None:
    """Display agent capabilities."""
    print("\nCapabilities:")
    print(f"   Streaming: {capabilities.get('streaming', False)}")
    print(f"   Push Notifications: {capabilities.get('pushNotifications', False)}")
    print(f"   State History: {capabilities.get('stateTransitionHistory', False)}")


def display_provider_info(provider: dict[str, Any]) -> None:
    """Display provider information."""
    print("\nProvider:")
    print(f"   Organization: {provider.get('organization', 'Unknown')}")
    print(f"   URL: {provider.get('url', 'Unknown')}")


def group_skills_by_type(skills: list[dict[str, Any]]) -> dict[str, list[dict[str, Any]]]:
    """Group skills by their type/plugin."""
    skill_groups = {
        'search': [],
        'file_ops': [],
        'system_ops': [],
        'other': []
    }

    for skill in skills:
        skill_id = skill.get('id', '').lower()

        if 'search' in skill_id:
            skill_groups['search'].append(skill)
        elif 'file' in skill_id or 'directory' in skill_id:
            skill_groups['file_ops'].append(skill)
        elif any(keyword in skill_id for keyword in ['system', 'command', 'working', 'hash']):
            skill_groups['system_ops'].append(skill)
        else:
            skill_groups['other'].append(skill)

    return skill_groups


def display_skill_group(title: str, skills: list[dict[str, Any]], max_display: int = 4) -> None:
    """Display a group of skills with consistent formatting."""
    if not skills:
        return

    print(f"\n   {title}:")

    for i, skill in enumerate(skills[:max_display]):
        name = skill.get('name', 'Unknown')
        skill_id = skill.get('id', 'no-id')
        description = skill.get('description', 'No description')

        # Format for better readability
        if len(description) > 60:
            description = description[:57] + "..."

        print(f"      • {name} ({skill_id})")
        print(f"        {description}")

        # Show example if available
        examples = skill.get('examples', [])
        if examples and i < 2:  # Only show examples for first 2 skills
            example = examples[0] if isinstance(examples[0], str) else str(examples[0])
            if len(example) > 50:
                example = example[:47] + "..."
            print(f"        Example: {example}")

    if len(skills) > max_display:
        print(f"        ... and {len(skills) - max_display} more")


def display_skills(skills: list[dict[str, Any]]) -> None:
    """Display all available skills grouped by type."""
    print(f"\nAvailable Skills ({len(skills)} total):")

    skill_groups = group_skills_by_type(skills)

    # Display each group
    display_skill_group("Web Search", skill_groups['search'], max_display=2)
    display_skill_group("File Operations", skill_groups['file_ops'], max_display=3)
    display_skill_group("System Operations", skill_groups['system_ops'], max_display=3)
    display_skill_group("Other Tools", skill_groups['other'], max_display=2)


def display_security_info(agent_card: dict[str, Any]) -> None:
    """Display security configuration."""
    print("\nSecurity Configuration:")

    security_schemes = agent_card.get('securitySchemes')
    if security_schemes:
        print(f"   Security Schemes: {list(security_schemes.keys())}")
    else:
        print("   Authentication: API Key based (X-API-Key header)")

    # Show security requirements from skills
    skills = agent_card.get('skills', [])
    scopes_found = set()

    for skill in skills:
        security = skill.get('security', [])
        for sec_req in security:
            if 'scopes' in sec_req:
                scopes_found.update(sec_req['scopes'])

    if scopes_found:
        print(f"   Required Scopes: {', '.join(sorted(scopes_found))}")


def display_communication_info(agent_card: dict[str, Any]) -> None:
    """Display communication capabilities."""
    print("\nCommunication:")
    print(f"   Input Modes: {', '.join(agent_card.get('defaultInputModes', ['text']))}")
    print(f"   Output Modes: {', '.join(agent_card.get('defaultOutputModes', ['text']))}")
    print(f"   Transport: {agent_card.get('preferredTransport', 'JSONRPC')}")


def display_insights(agent_card: dict[str, Any]) -> None:
    """Display key insights about the agent."""
    capabilities = agent_card.get('capabilities', {})
    skills = agent_card.get('skills', [])

    print("\nKey Insights:")

    if capabilities.get('streaming'):
        print("   ✅ Real-time streaming support enables live research feedback")

    if len(skills) >= 10:
        print(f"   ✅ Rich toolset with {len(skills)} capabilities for comprehensive research")

    if any('search' in s.get('id', '') for s in skills):
        print("   ✅ Web search integration for real-time financial data")

    if any('file' in s.get('id', '') for s in skills):
        print("   ✅ File operations enable persistent research storage")

    # Check for financial research relevance
    description = agent_card.get('description', '').lower()
    if 'financial' in description or 'research' in description:
        print("   ✅ Specialized for financial research and market analysis")

# Execute the agent card discovery directly (notebook-friendly)
print("Agent Card Discovery Test")
print("=" * 50)

try:
    # Fetch the agent card
    agent_card = fetch_agent_card()

    # Display all information
    display_agent_info(agent_card)
    display_capabilities(agent_card.get('capabilities', {}))
    display_provider_info(agent_card.get('provider', {}))
    display_skills(agent_card.get('skills', []))
    display_security_info(agent_card)
    display_communication_info(agent_card)
    display_insights(agent_card)

    print("\n" + "=" * 50)
    print("The agent card provides a standardized way to discover agent capabilities")
    print("This endpoint follows the A2A protocol specification for agent discovery")
    print("Use this information to understand what the agent can do before making requests")

except Exception as e:
    print(f"❌ Agent card discovery failed: {e}")
    print("\nTroubleshooting:")
    print("   • Ensure the AgentUp server is running on http://127.0.0.1:8000")
    print("   • Check that the agent configuration is valid")
    print("   • Verify network connectivity to the agent server")

# Perform Research

Ok Let's perform the research!

### Cryptocurrency Market Trends for 2025

In [ ]:
# Initialize the agent client
agent = FinancialResearchAgent()

# Get environment-appropriate path for file saving
try:
    ENV_TYPE, PROJ_PATH = detect_environment()
    if ENV_TYPE == 'colab':
        save_path = "/content/proj-folder/research_data"
    else:
        save_path = "./research_data"
    print(f"Research files will be saved to: {save_path}")
except NameError:
    save_path = "./research_data"  # Fallback
    print("Using fallback save path: research_data")

# Research cryptocurrency market trends with environment-aware file saving
print("\n🔍 Cryptocurrency Research with Streaming:")
print("=" * 50)
print("Watch the agent work in real-time!")
print()

crypto_research = agent.research_topic(f"Bitcoin and Ethereum market trends 2026 to 2030. Save a full in-depth research document of all findings and commentary to {save_path}/crypto_analysis.md")

if crypto_research.get('error'):
    print(f"\n❌ Research failed: {crypto_research['error']}")
else:
    print("\n✅ Research completed successfully!")
    print(f"📊 Response status: {crypto_research.get('success', 'Unknown')}")

print("\n" + "=" * 50)

### Tech Stocks Analysis for 2025

In [ ]:
# Research tech stock performance with environment-aware file saving
try:
    ENV_TYPE, PROJ_PATH = detect_environment()
    if ENV_TYPE == 'colab':
        save_path = "/content/proj-folder/research_data"
    else:
        save_path = "./research_data"
except NameError:
    save_path = "./research_data"  # Fallback

tech_stocks = agent.research_topic(f"NVIDIA Apple Microsoft Google stock performance 2025 AI impact.")

print("📈 Tech Stocks Research Results:")
print("=" * 50)

if tech_stocks.get('error'):
    print(f"❌ Research failed: {tech_stocks['error']}")
else:
    result = tech_stocks.get('result', {})
    if 'content' in result:
        print(result['content'])
    else:
        print(json.dumps(tech_stocks, indent=2))

print("\n" + "=" * 50)

### Market Sector Analysis

Now let's analyze a specific market sector with iterative refinement.

In [ ]:
try:
    ENV_TYPE, PROJ_PATH = detect_environment()
    if ENV_TYPE == 'colab':
        save_path = "/content/proj-folder/research_data"
    else:
        save_path = "research_data"
except NameError:
    save_path = "research_data"  # Fallback

# Research renewable energy sector
sector_analysis = agent.research_topic(f"renewable energy stocks solar wind ESG investing trends 2025.")

print("🌱 Renewable Energy Sector Analysis:")
print("=" * 50)

if sector_analysis.get('error'):
    print(f"❌ Research failed: {sector_analysis['error']}")
else:
    result = sector_analysis.get('result', {})
    if 'content' in result:
        print(result['content'])
    else:
        print(json.dumps(sector_analysis, indent=2))

print("\n" + "=" * 50)

## Examining Generated Research Files

Let's check what research files our agent has created and examine their contents.

In [ ]:
import datetime
from pathlib import Path

# list files in the research directory
research_dir = Path("./research_data")

if research_dir.exists():
    files = list(research_dir.glob("*.txt")) + list(research_dir.glob("*.md")) + list(research_dir.glob("*.json"))

    print(f"📁 Research Files Created ({len(files)} files):")
    print("=" * 50)

    for file_path in files:
        print(f"📄 {file_path.name}")
        print(f"   Size: {file_path.stat().st_size} bytes")
        print(f"   Modified: {datetime.datetime.fromtimestamp(file_path.stat().st_mtime)}")

        # Show first few lines of each file
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read(500)  # First 500 characters
                print(f"   Preview: {content[:200]}...")
        except Exception as e:
            print(f"   Error reading file: {e}")

        print()

else:
    print("❌ Research directory not found")

## Advanced Iterative Research Workflow

Let's demonstrate a more complex iterative workflow where the agent:
1. Performs initial research
2. Analyzes gaps in the information
3. Conducts follow-up searches
4. Generates a comprehensive final report

In [ ]:
# Complex iterative research workflow with streaming
async def complex_research_demo():
    # Get environment-appropriate path for file saving
    try:
        ENV_TYPE, PROJ_PATH = detect_environment()
        if ENV_TYPE == 'colab':
            save_path = "/content/proj-folder/research_data"
        else:
            save_path = "research_data"
        print(f"Research files will be saved to: {save_path}")
    except NameError:
        save_path = "research_data"  # Fallback
        print("Using fallback save path: research_data")
    
    prompt = f"""
Conduct a comprehensive multi-stage research analysis on "Federal Reserve interest rate policy impact on financial markets 2024".

Please follow this iterative workflow:

STAGE 1: Initial Information Gathering
- Search for recent Fed policy announcements and decisions
- Gather data on current interest rates and recent changes
- Save initial findings to '{save_path}/fed_policy_initial.md'

STAGE 2: Market Impact Analysis
- Based on Stage 1 findings, search for specific market reactions
- Focus on bond markets, stock markets, and currency impacts
- Look for expert analysis and predictions
- Save market analysis to '{save_path}/fed_policy_market_impact.md'

STAGE 3: Sector-Specific Effects
- Research how Fed policy affects different sectors (banking, real estate, tech)
- Find specific examples of companies or sectors most impacted
- Save sector analysis to '{save_path}/fed_policy_sectors.md'

STAGE 4: Comprehensive Report
- Synthesize all previous research into a final comprehensive report
- Include executive summary, key findings, and future outlook
- Save final report to '{save_path}/fed_policy_comprehensive_report.md'

Please execute each stage sequentially and provide progress updates.
"""

    print("\n🔍 Advanced Iterative Research Workflow with Streaming:")
    print("=" * 60)
    print("This will demonstrate the complete iterative research process...")
    print()

    # Stream the complex research
    async def display_progress(chunk):
        print(chunk, end='', flush=True)

    result = await agent.send_message_stream(prompt, display_progress)

    print(f"\n{'-' * 60}")
    if result.get('error'):
        print(f"❌ Research failed: {result['error']}")
    else:
        print("✅ Multi-stage research workflow completed!")

    return result

# Execute the complex research
complex_research = await complex_research_demo()
print("\n" + "=" * 60)

## Data Analysis and Visualization

Now let's analyze the research data we've collected and create some visualizations. This demonstrates how the agent can combine web search with local data processing.

In [ ]:
# Ask agent to create a data analysis with visualizations
analysis_request = await agent.send_message("""
Create a Python script that analyzes our research findings and generates visualizations.

Please:
1. Read all the research files we've created
2. Extract key data points and trends
3. Create a Python analysis script that includes:
   - Text analysis of research findings
   - Simple data visualizations (charts/graphs)
   - Summary statistics
4. Save the script as 'research_analysis.py'
5. Execute the script to generate the analysis

Focus on extracting quantitative data where possible (percentages, dates, financial figures).
""")

print("📊 Research Data Analysis:")
print("=" * 50)

if analysis_request.get('error'):
    print(f"❌ Analysis failed: {analysis_request['error']}")
else:
    result = analysis_request.get('result', {})
    if 'content' in result:
        print(result['content'])
    else:
        print(json.dumps(analysis_request, indent=2))

## Plugin Integration Demonstration

Let's specifically demonstrate how the two plugins work together by having the agent:
1. Search for specific financial data (Brave plugin)
2. Process and store that data (Systools plugin)
3. Create structured reports

In [ ]:
# Demonstrate explicit plugin integration
plugin_demo = await agent.send_message("""
Demonstrate the integration between agentup_systools and agentup_brave plugins:

1. Use Brave Search to find the latest stock prices for Apple (AAPL), Google (GOOGL), and Microsoft (MSFT)
2. Use system tools to create a structured CSV file with the data
3. Create a directory called 'stock_data' for organized storage
4. Save the CSV file as 'top_tech_stocks.csv'
5. Create a summary report explaining what you found and saved

Please show each step clearly and explain how the plugins work together.
""")

print("🔧 Plugin Integration Demonstration:")
print("=" * 50)

if plugin_demo.get('error'):
    print(f"❌ Demo failed: {plugin_demo['error']}")
else:
    result = plugin_demo.get('result', {})
    if 'content' in result:
        print(result['content'])
    else:
        print(json.dumps(plugin_demo, indent=2))

## Error Handling and Security Demonstration

Let's test the agent's error handling and security features by attempting operations that might fail or require different permissions.

In [ ]:
import nest_asyncio

# Test error handling and security
print("🔒 Testing Error Handling and Security:")
print("=" * 50)



nest_asyncio.apply()

async def test_error_handling():
	# Test 2: File operation outside workspace (should be restricted)
	restricted_file = await agent.send_message("Create a file at /etc/restricted_file.txt")
	print("Test 2 - Restricted file location:")
	print(f"Result: {restricted_file.get('error', 'Success')}\n")

	# Test 3: Large file handling
	large_file_test = await agent.send_message("Try to create a very large file (20MB) to test size limits")
	print("Test 3 - File size limits:")
	print(f"Result: {large_file_test.get('error', 'Success')}\n")

await test_error_handling()

## Final Research Summary

Let's have the agent create a comprehensive summary of all the research we've conducted in this session.

In [ ]:
# Generate comprehensive session summary
session_summary = await agent.send_message("""
Create a comprehensive summary of all the financial research we've conducted in this session.

Please:
1. list all the research files you've created
2. Summarize the key findings from each research topic
3. Identify common themes and insights across all research
4. Create a final report called 'session_summary_report.md'
5. Include recommendations for further research

This should serve as a complete overview of our financial research session.
""")

print("📋 Session Summary:")
print("=" * 50)

if session_summary.get('error'):
    print(f"❌ Summary failed: {session_summary['error']}")
else:
    result = session_summary.get('result', {})
    if 'content' in result:
        print(result['content'])
    else:
        print(json.dumps(session_summary, indent=2))

## Key Learnings and Takeaways

This notebook demonstrated several important concepts:

### 1. **Plugin Architecture**
- AgentUp's plugin system enables modular functionality
- Plugins declare their capabilities and required scopes
- Framework handles security enforcement automatically

### 2. **Iterative Agent Design**
- Agents can perform multi-step workflows
- Each step builds on previous findings
- Information is preserved between iterations

### 3. **Security-First Approach**
- Scope-based authorization provides fine-grained control
- Plugins must declare required permissions
- Framework fails securely when permissions are missing

### 4. **A2A Protocol Integration**
- Standardized communication format
- JSON-RPC 2.0 for reliable message exchange
- Authentication via API keys

### 5. **Production Readiness**
- Comprehensive error handling
- Resource management and cleanup
- Performance monitoring capabilities
- Structured logging and audit trails

## Next Steps

To extend this example:
- Add more specialized financial data sources
- Implement real-time data streaming
- Create automated report scheduling
- Add data visualization dashboards
- Integrate with financial APIs (Alpha Vantage, Yahoo Finance)
- Implement portfolio analysis capabilities

## Resources

- [AgentUp Documentation](https://docs.agentup.dev)
- [Plugin Development Guide](https://docs.agentup.dev/plugin-development/)

## Stay Updated

AgentUp development is moving at a fast pace 🏃‍♂️, for a great way to follow the project and to be instantly notified of new releases, **Star the [AgentUp Repository](https://github.com/RedDotRocket/AgentUp)**

<img src="https://raw.githubusercontent.com/RedDotRocket/AgentUp/refs/heads/main/assets/star.gif" width="20%" height="20%"/>
